### MLP - Multi Layer Perceptron- Basic Neural Network ###

In [ ]:
!pip install torch torchvision

In [ ]:
import torch
from torch import nn
from torch import optim
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import numpy as np

In [ ]:
# Define the MLP model
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.layers(x)
        return x

In [ ]:
# Load and transform the data
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_set = MNIST(root='./data', train=True, download=True, transform=transform)
test_set = MNIST(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

In [ ]:
# Initialize the model, loss function and optimizer
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = MLP().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
# Train the model
for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {running_loss/len(train_loader)}")

print('Finished Training')


In [ ]:
import matplotlib.pyplot as plt
import random

# Define a function to visualize the image and prediction
def visualize_prediction(img, prediction, ax1, ax2):
    # Convert prediction to probabilities using softmax
    probabilities = nn.functional.softmax(prediction, dim=0).detach().numpy()

    # Plot the image and the prediction
    ax1.imshow(img.squeeze(), cmap='gray')
    ax1.axis('off')
    ax2.bar(np.arange(10), probabilities)
    ax2.set_xlabel('Digit Class')
    ax2.set_ylabel('Predicted Probability')
    ax2.set_xticks(np.arange(10))

# Create figure for plotting
fig, axs = plt.subplots(nrows=10, ncols=2, figsize=(10, 30))

# Make a prediction using the model and visualize it for 10 random images from the test set
model.eval()
with torch.no_grad():
    for i in range(10):
        index = random.randint(0, len(test_set)-1)  # select a random index
        image, label = test_set[index]
        image = image.to(device)

        # Add an extra dimension to the image tensor and make a prediction
        prediction = model(image.unsqueeze(0))

        # Move the image and prediction to cpu for visualization
        image = image.cpu()
        prediction = prediction.cpu()

        # Remember to get the first element of the prediction tensor
        prediction = prediction[0]

        # Visualize the image and prediction
        visualize_prediction(image, prediction, axs[i, 0], axs[i, 1])

# Show the plot
plt.tight_layout()
plt.show()
